# GigaChat

В этом файле показано, как можно работать с GigaChat.

Для начала работы вам нужно будет получить ключи доступа для работы с GigaChat. Подробнее об этом вы можете прочитать в [этом документе](docs/extras/integrations/chat/README.md)

In [4]:
from langchain.chat_models import GigaChat
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [6]:
chat = GigaChat(credentials=...)

В ячейке выше показано, как подключить к GigaChat. Вы также можете хранить авторизационные данные в переменной окружения `GIGACHAT_CREDENTIALS`.

In [15]:
messages = [
    SystemMessage(
        content="Ты полезный ассистент, который умеет переводить русский на английский."
    ),
    HumanMessage(content="Переведи это предложение. Я люблю программирование."),
]
chat(messages)

AIMessage(content='I love programming.', additional_kwargs={}, example=False)

Вы можете использовать шаблонизацию с помощью `MessagePromptTemplate`. Вы можете создать `ChatPromptTemplate` из одного или нескольких `MessagePromptTemplates`. Вы можете использовать метод `format_prompt` класса `ChatPromptTemplate` — это возвращает `PromptValue`, который вы можете преобразовать в строку или объект сообщения, в зависимости от того, хотите ли вы использовать отформатированное значение как входные данные для модели llm или чат-модели.

Для удобства есть метод `from_template`, предоставленный в шаблоне. Если вы хотите использовать этот шаблон, это будет выглядеть примерно так:

In [11]:
template = "Ты полезный ассистент, который умеет переводить {input_language} на {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [14]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
chat(
    chat_prompt.format_prompt(
        input_language="английский",
        output_language="русский",
        text="Translate this sentense. I love programming.",
    ).to_messages()
)

AIMessage(content='Я люблю программирование.', additional_kwargs={}, example=False)

## Потоковое получение ответа
GigaChat может возвращать промежуточные результаты генерации ответа пользователю.

Для этого нужно создать класс-наследник `BaseCallbackHandler`

In [12]:
from langchain.callbacks.base import BaseCallbackHandler

class StreamHandler(BaseCallbackHandler):
    def __init__(self, initial_text=""):
        pass

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{token} -", end="", flush=True)

Гигачат нужно инициализировать в режиме стриминга:

In [13]:
chat = GigaChat(credentials=..., streaming=True, callbacks=[StreamHandler()])

Теперь, при вызове генерации можно увидеть, как ответ модели последовательно выводится пользователю.

In [14]:
chat([HumanMessage(content="Напиши краткое содержание романа Евгений Онегин")])

«Евгений Онегин» — роман в стихах, написанный Александром Сергеевичем Пушкиным. Произведение было написано в период с 1823 по 1831 годы и впервые опубликовано в журнале «Современник».

Роман рассказывает о жизни двух друзей — Евгения Онегина и Владимира Ленского. Они проводят много - времени - вместе -, - де -лятся - своими - мы -сля -ми - и - чувства -ми -. - Однако - между - ними - возникает - конфликт - из -- -за - любви - к - одной - женщи -не - — - Оль -ге - Ла -ри -ной -. - В - результате - ду -э -ли - Владимир - Л -ен -ский - поги -ба -ет -, - а - Евгений - О -не -гин - поки -дает - Россию -. -
 -
 -Ч -ере -з - несколько - лет - Евгений - возвра -щается - в - Россию - и - начинает - уха -живать - за - Оль -гой -. - Он - призна -ется - ей - в - любви -, - но - она - отвер -гает - его -. - В - конце - концов - Ольга - выходит - заму -ж - за - князя - Гре -чина -. -
 -
 -О -не -гин - разоча -рован - и - у -яз -влен - таким - ис -ходом - событий -. - Он - решает - поки -нуть - Росси

AIMessageChunk(content='«Евгений Онегин» — роман в стихах, написанный Александром Сергеевичем Пушкиным. Произведение было написано в период с 1823 по 1831 годы и впервые опубликовано в журнале «Современник».\n\nРоман рассказывает о жизни двух друзей — Евгения Онегина и Владимира Ленского. Они проводят много времени вместе, делятся своими мыслями и чувствами. Однако между ними возникает конфликт из-за любви к одной женщине — Ольге Лариной. В результате дуэли Владимир Ленский погибает, а Евгений Онегин покидает Россию.\n\nЧерез несколько лет Евгений возвращается в Россию и начинает ухаживать за Ольгой. Он признается ей в любви, но она отвергает его. В конце концов Ольга выходит замуж за князя Гречина.\n\nОнегин разочарован и уязвлен таким исходом событий. Он решает покинуть Россию навсегда и отправляется в Европу.')